In [0]:
from pyspark.sql.functions import *
from datetime import datetime

date_ = datetime.utcnow().strftime("%Y-%m-%d")

bronze_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/githublogs/bronze/{date_}"
silver_path = f"abfss://projectfiles@practicestorageacc0.dfs.core.windows.net/githublogs/silver/{date_}"
df_bronze = spark.readStream.format("delta").load(bronze_path)

In [0]:
df_silver = df_bronze \
    .filter("event_type is not null and timestamp is not null and actor is not null")  \
    .withColumn("event_time", to_timestamp(col("timestamp"))) \
    .withColumn("event_type", trim(lower(col("event_type")))) \
    .withColumn("repo", trim(col("repo"))) \
    .withColumn("actor", trim(col("actor")))

df_silver = df_silver \
    .withColumn("org", split(col("repo"), "/").getItem(0)) \
    .withColumn("repo_name", split(col("repo"), "/").getItem(1))

In [0]:
query = (
    df_silver.writeStream
    .format("delta")
    .option("checkpointLocation", f"/temp/checkpoints/githublogs/silver/{date_}")
    .option("path", silver_path)
    .trigger(availableNow=True)
    .start()
)
query.awaitTermination()